In [1]:
import os
# Changing current working directory
os.chdir('/Users/kai23/Library/Mobile Documents/com~apple~CloudDocs/MSc Data Science/Skills for DS/Skills Assessment 2')
print("Working Directory Changed To:", os.getcwd())

Working Directory Changed To: /Users/kai23/Library/Mobile Documents/com~apple~CloudDocs/MSc Data Science/Skills for DS/Skills Assessment 2


In [2]:
import pandas as pd
# Loading the CSV file
authors = pd.read_csv('./data/authors.APOE.csv')

authors.info()
authors.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56892 entries, 0 to 56891
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   PMID               56892 non-null  int64 
 1   AuthorN            56892 non-null  int64 
 2   AuthorForename     56865 non-null  object
 3   AuthorLastname     56892 non-null  object
 4   AuthorInitials     56815 non-null  object
 5   AuthorAffiliation  35701 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.6+ MB


,PMID,AuthorN,AuthorForename,AuthorLastname,AuthorInitials,AuthorAffiliation
0,10545039,1,K E,Morrison,KE,"Neurosciences Group, Institiute of Molecular M..."
1,10545039,2,G,Steers,G,NaN
2,10545039,3,V,Dubowitz,V,NaN
3,10558867,1,M L,Brandi,ML,"Department of Clinical Physiopathology, Univer..."
4,10558867,2,L,Becherini,L,NaN


In [3]:
# Identifying missing values
print(authors.isnull().sum())

PMID                     0
AuthorN                  0
AuthorForename          27
AuthorLastname           0
AuthorInitials          77
AuthorAffiliation    21191
dtype: int64


In [4]:
# Using placeholders for missing values of names/initials
authors['AuthorForename'] = authors['AuthorForename'].fillna('Unknown')
authors['AuthorInitials'] = authors['AuthorInitials'].fillna('N/A')

In [5]:
# Duplicating "AuthorAffiliation" for every author of that paper
authors['AuthorAffiliation'] = authors.groupby('PMID')['AuthorAffiliation'].transform(lambda x: x.ffill().bfill())

/var/folders/1c/pv56fs9s22lgvn6pw288swth0000gn/T/ipykernel_64578/969249506.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  authors['AuthorAffiliation'] = authors.groupby('PMID')['AuthorAffiliation'].transform(lambda x: x.ffill().bfill())


In [6]:
# Adding a column to show different number of authors per paper
authors['AuthorsPerPaper'] = authors.groupby('PMID')['AuthorN'].transform('max')

In [7]:
# Creating a new column for full names
authors['AuthorFullName'] = authors['AuthorForename'] + ' ' + authors['AuthorLastname']

In [8]:
import re

# Defining a function to clean affiliations column
def clean_affiliation(affiliation):
    if pd.isna(affiliation):
        return "Unknown"
    
    # Removing area codes and specific country names
    cleaned = re.sub(r'\s*\d{5}(-\d{4})?|,?\s*USA|,?\s*UK|,?\s*China|,?\s*Sweden', '', affiliation, flags=re.IGNORECASE)
    
    # Removing extra commas and spaces
    cleaned = re.sub(r',+', ',', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip(', ')
    
    return cleaned

# Applying the function to the AuthorAffiliation column
authors['AuthorAffiliation'] = authors['AuthorAffiliation'].apply(clean_affiliation)

In [9]:
# Removing leading/trailing whitespace and ensuring consistent capitalisation
authors['AuthorAffiliation'] = authors['AuthorAffiliation'].str.strip()
authors['AuthorAffiliation'] = authors['AuthorAffiliation'].str.title()

# Creating dictionary to standardise names
affiliation_standardise = {
    "Department Of Neuroscience, Mayo Clinic College Of Medicine, Jacksonville, Fl,.": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Neuroscience, Mayo Clinic, Jacksonville, Fl,. Zjing.0321@Gmail.Com.": "Mayo Clinic, Jacksonville, Florida",
    "Cytometry And Cell Imaging Laboratory, Mayo Clinic, Jacksonville, Fl.": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Quantitative Health Sciences, Mayo Clinic, Jacksonville, Fl,.": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Quantitative Health Sciences, Mayo Clinic, Jacksonville, Fl,.": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Neuroscience, Mayo Clinic, Jacksonville, Fl,. Guojun.Bu@Molecularneurodegeneration.Org.": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Neuroscience, Mayo Clinic, Jacksonville, Fl. Liu.Chiachenjen@Gmail.Com.": "Mayo Clinic, Jacksonville, Florida",
    "4500 San Pablo Rd, Mayo Clinic, Jacksonville, Fl.": "Mayo Clinic, Jacksonville, Florida",    
    "Department Of Neuroscience, Mayo Clinic, Jacksonville, Florida": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Neuroscience, Mayo Clinic, Jacksonville, Florida.": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Neuroscience, Mayo Clinic, Jacksonville, Fl.": "Mayo Clinic, Jacksonville, Florida",
    "Department Of Radiology, Mayo Clinic, Rochester, Mn.": "Mayo Clinic, Rochester, Minnesota",
}

# Applying the dictionary
authors['AuthorAffiliation'] = authors['AuthorAffiliation'].replace(affiliation_standardise)

In [10]:
# Saving cleaned csv to outputs folder
authors.to_csv('./outputs/authors_cleaned.csv', index=False)